In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SeparableConv2D, DepthwiseConv2D, AveragePooling2D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/MRNet-v1.0"

In [ ]:
def read_data(data_type, scan_type):
  list = []
  for filename in sorted(os.listdir("/content/MRNet-v1.0/"+data_type+"/"+scan_type)):
    if(filename != ".DS_Store"):
      instance = np.load("/content/MRNet-v1.0/"+data_type+"/"+scan_type+"/" + filename)
      mid = len(instance) // 2
      instance = np.array([ instance[mid-1] , instance[mid] ,instance[mid+1]])
      list.append(instance)
  data = np.asarray(list)
  data = np.moveaxis(data, 1, 3)
  print(scan_type+data_type, data.shape)
  return data

def read_labels(data_type, scan_type):
  path = "/content/MRNet-v1.0/"+data_type+"-"+scan_type + ".csv"
  df=pd.read_csv(path, sep=',',header=None)
  labels = df.values
  labels = labels[:,1]
  print(scan_type+data_type, labels.shape)
  return labels


def read_data_singles(data_type, scan_type, label_scan_type):
  list = []
  path = "/content/MRNet-v1.0/"+data_type+"-"+label_scan_type + ".csv"
  df=pd.read_csv(path, sep=',',header=None)
  labels = df.values
  labels = labels[:,1]
  single_labels = []
  i=0
  for filename in sorted(os.listdir("/content/MRNet-v1.0/"+data_type+"/"+scan_type)):
    if(filename != ".DS_Store"):
      instance = np.load("/content/MRNet-v1.0/"+data_type+"/"+scan_type+"/" + filename)
      for image in instance:
        list.append(np.array(image))
        single_labels.append(labels[i])
      i += 1
  data = np.asarray(list)
  data = data[:, :, :, np.newaxis]
  single_labels = np.asarray(single_labels)
  print("single")
  print(scan_type+data_type, data.shape)
  print(label_scan_type+data_type, single_labels.shape)
  return data, single_labels

single_sagittal_train_x, single_saggital_abnormal_train_y = read_data_singles("train","sagittal","abnormal")
single_sagittal_test_x, single_saggital_abnormal_test_y = read_data_singles("valid","sagittal","abnormal")

# print("Data: ")

# sagittal_train_x = read_data("train","sagittal")
# axial_train_x = read_data("train","axial")
# coronal_train_x = read_data("train","coronal")

# sagittal_test_x = read_data("valid","sagittal")
# axial_test_x = read_data("valid","axial")
# coronal_test_x = read_data("valid","coronal")

# print("Labels: ")
# abnormal_train_y = read_labels('train','abnormal')
# acl_train_y = read_labels('train','acl')
# meniscus_train_y = read_labels('train','meniscus')

# abnornmal_test_y = read_labels('valid','abnormal')
# acl_test_y = read_labels('valid','acl')
# meniscus_test_y = read_labels('valid','meniscus')

single
sagittaltrain (34370, 256, 256, 1)
abnormaltrain (34370,)
single
sagittalvalid (3663, 256, 256, 1)
abnormalvalid (3663,)


In [ ]:
#VGG-16 model
vgg_model = Sequential()
vgg_model.add(Conv2D(64, (3,3), activation='relu', padding="same", input_shape=(256, 256, 1)))
vgg_model.add(Conv2D(64, (3,3), activation='relu', padding="same"))
vgg_model.add(MaxPooling2D(pool_size=(2, 2)))
vgg_model.add(Conv2D(128, (3,3), activation='relu', padding="same"))
vgg_model.add(Conv2D(128, (3,3), activation='relu', padding="same"))
vgg_model.add(MaxPooling2D(pool_size=(2, 2)))
vgg_model.add(Conv2D(256, (3,3), activation='relu', padding="same"))
vgg_model.add(Conv2D(256, (3,3), activation='relu', padding="same"))
vgg_model.add(Conv2D(256, (3,3), activation='relu', padding="same"))
vgg_model.add(MaxPooling2D(pool_size=(2, 2)))
vgg_model.add(Conv2D(512, (3,3), activation='relu', padding="same"))
vgg_model.add(Conv2D(512, (3,3), activation='relu', padding="same"))
vgg_model.add(Conv2D(512, (3,3), activation='relu', padding="same"))
vgg_model.add(MaxPooling2D(pool_size=(2, 2)))
vgg_model.add(Conv2D(512, (3,3), activation='relu', padding="same"))
vgg_model.add(Conv2D(512, (3,3), activation='relu', padding="same"))
vgg_model.add(Conv2D(512, (3,3), activation='relu', padding="same"))
vgg_model.add(MaxPooling2D(pool_size=(2, 2)))

vgg_model.add(Flatten())
vgg_model.add(Dense(4096, activation='relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(Dense(4096, activation='relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
vgg_model.compile(loss='binary_crossentropy', optimizer= sgd, metrics=['accuracy'])
vgg_model.fit(single_sagittal_train_x, single_saggital_abnormal_train_y, batch_size=32, epochs=15, validation_split=0.1, 
              verbose=1,shuffle=True)
vgg_model.evaluate(single_sagittal_test_x, single_saggital_abnormal_test_y, batch_size=32)

Epoch 1/15
967/967 [==============================] - 582s 602ms/step - loss: 0.3853 - accuracy: 0.8389 - val_loss: 0.3815 - val_accuracy: 0.8426
Epoch 2/15
967/967 [==============================] - 582s 601ms/step - loss: 0.3452 - accuracy: 0.8500 - val_loss: 0.3725 - val_accuracy: 0.8408
Epoch 3/15
967/967 [==============================] - 582s 602ms/step - loss: 0.3272 - accuracy: 0.8571 - val_loss: 0.3541 - val_accuracy: 0.8382
Epoch 4/15
967/967 [==============================] - 582s 602ms/step - loss: 0.3133 - accuracy: 0.8645 - val_loss: 0.3426 - val_accuracy: 0.8458
Epoch 5/15
967/967 [==============================] - 585s 605ms/step - loss: 0.3002 - accuracy: 0.8681 - val_loss: 0.3528 - val_accuracy: 0.8408
Epoch 6/15
967/967 [==============================] - 585s 605ms/step - loss: 0.2851 - accuracy: 0.8757 - val_loss: 0.3605 - val_accuracy: 0.8475
Epoch 7/15
967/967 [==============================] - 584s 604ms/step - loss: 0.2695 - accuracy: 0.8804 - val_loss: 0.3413 -

[0.8407562375068665, 0.8443898558616638]

In [ ]:
#MobileNet model
mobile_model = Sequential()
mobile_model.add(Conv2D(32, (3,3), strides=(2,2), activation='relu', padding = "same", input_shape=(256, 256, 1)))
mobile_model.add(DepthwiseConv2D((3,3), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(64, (1,1), activation='relu', padding = "same"))
mobile_model.add(DepthwiseConv2D((3,3), strides=(2,2), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(128, (1,1), activation='relu', padding = "same"))
mobile_model.add(DepthwiseConv2D((3,3), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(128, (1,1), activation='relu', padding = "same"))
mobile_model.add(DepthwiseConv2D((3,3), strides=(2,2), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(256, (1,1), activation='relu', padding = "same"))
mobile_model.add(DepthwiseConv2D((3,3), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(256, (1,1), activation='relu', padding = "same"))
mobile_model.add(DepthwiseConv2D((3,3), strides=(2,2), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(512, (1,1), activation='relu', padding = "same"))

mobile_model.add(DepthwiseConv2D((3,3), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(512, (1,1), activation='relu', padding = "same"))
mobile_model.add(DepthwiseConv2D((3,3), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(512, (1,1), activation='relu', padding = "same"))
mobile_model.add(DepthwiseConv2D((3,3), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(512, (1,1), activation='relu', padding = "same"))
mobile_model.add(DepthwiseConv2D((3,3), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(512, (1,1), activation='relu', padding = "same"))
mobile_model.add(DepthwiseConv2D((3,3), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(512, (1,1), activation='relu', padding = "same"))


mobile_model.add(DepthwiseConv2D((3,3), strides=(2,2), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(1024, (1,1), activation='relu', padding = "same"))
mobile_model.add(DepthwiseConv2D((3,3), depth_multiplier=1, activation='relu', padding = "same"))
mobile_model.add(Conv2D(1024, (1,1), activation='relu', padding = "same"))
mobile_model.add(AveragePooling2D((8,8)))
mobile_model.add(Flatten())
mobile_model.add(Dense(1000, activation='relu'))
mobile_model.add(Dropout(0.5))
mobile_model.add(Dense(1, activation='sigmoid'))

mobile_model.compile(loss='binary_crossentropy', optimizer= "adam", metrics=['accuracy'])
mobile_model.fit(single_sagittal_train_x, single_saggital_abnormal_train_y, batch_size=32, epochs=25, validation_split=0.1, verbose=1,shuffle=True)
mobile_model.evaluate(single_sagittal_test_x, single_saggital_abnormal_test_y, batch_size=32)

Epoch 1/25
967/967 [==============================] - 198s 205ms/step - loss: 0.4547 - accuracy: 0.8337 - val_loss: 0.4486 - val_accuracy: 0.8350
Epoch 2/25
967/967 [==============================] - 197s 203ms/step - loss: 0.4526 - accuracy: 0.8344 - val_loss: 0.4468 - val_accuracy: 0.8350
Epoch 3/25
967/967 [==============================] - 197s 204ms/step - loss: 0.4513 - accuracy: 0.8344 - val_loss: 0.4470 - val_accuracy: 0.8350
Epoch 4/25
967/967 [==============================] - 196s 203ms/step - loss: 0.4516 - accuracy: 0.8344 - val_loss: 0.4469 - val_accuracy: 0.8350
Epoch 5/25
967/967 [==============================] - 195s 201ms/step - loss: 0.4506 - accuracy: 0.8344 - val_loss: 0.4477 - val_accuracy: 0.8350
Epoch 6/25
967/967 [==============================] - 196s 202ms/step - loss: 0.4506 - accuracy: 0.8344 - val_loss: 0.4468 - val_accuracy: 0.8350
Epoch 7/25
967/967 [==============================] - 195s 202ms/step - loss: 0.4502 - accuracy: 0.8344 - val_loss: 0.4466 -

[0.48937174677848816, 0.8099918365478516]

In [ ]:
stacked_train_y = np.vstack((abnormal_train_y,acl_train_y))
stacked_train_y = np.vstack((stacked_train_y,meniscus_train_y)).T

stacked_test_y = np.vstack((abnornmal_test_y,acl_test_y))
stacked_test_y = np.vstack((stacked_test_y,meniscus_test_y)).T